In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import networkx as nx
import json
import os
import base64

# Muhaddithat App

This notebook contains all application code for the Muhaddithat dashboard with documentation in markdown. We also have the same code without Jupyter formatting in the file `app.py`, which can be used for deployment on a server. _NOTE: app.py is not yet updated to the latest layout for the app. This file, app.ipynb contains the latest updates._

To start the app, click 'run all cells' and visit http://127.0.0.1:8050/ in your web browser.
Alternatively, you can open a terminal window, navigate to the project directory, and run `python app.py` *(note: `app.y` file has not yet been updated)*. If you use the latter method, make sure you are within a Python environment with all requisite packages installed.

In [2]:
app = JupyterDash(__name__)

## Data
Opening the files corresponding to each of the selected narrators

In [3]:
#filename = 'data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-3_SEEDS-53-54-56-59-69-70-71-84-244-2802-10526-10737-11039-11457.cyjs'


# set initial/default data to Narrator ID 53 (Aishah bint Abi Bakr)
with open('data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-53.cyjs', 'r') as f:
    data = json.load(f)

In [4]:
#data = data_53 # set default/initial data being displayed to Aishah bint Abi Bakr's
currentNarrator = 53 # variable to keep track of the current narrator being displayed

In [5]:
# Open the biographies and hadiths written/selected for each of the narrators

with open('data/bios.json', encoding='utf8') as f:
    bios = json.load(f)
    
with open('data/hadith.json', encoding='utf8') as f:
    hadiths = json.load(f)

In [6]:
# testing
# hadiths[0]['NarrationChain']
# print(bios[0]['Biography'])

## Styling

In [7]:
# stylesheet for the Cytoscape graph

default_stylesheet = [
    {
        'selector': '[gender="m"]',
        'style': {
            'background-color': '#7b7b7b',
            'background-opacity': '0.5',
            'width':'0.01px',
            'height':'0.01px',
            'padding':'3px'
        }
    },
    {
        'selector':'[gender="f"]',
        'style':{
            'background-color':'#ff0000',
            'width':'0.2px',
            'height':'0.2px',
            'padding':'3px'
        }        
    },
    {
        'selector':'[id="1"]',
        'style':{
            'background-color':'#8fce00',
            'width':'5px',
            'height':'5px',
            'background-opacity':'1'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    }    
]

## Layout

The following section specifies the layout of the app (header, sidebars, graph component, etc.). 

Resources on tabs:
- https://dash.plotly.com/dash-core-components#tabs
- https://dash.plotly.com/dash-core-components/tabs

Layout examples/resources:
- https://dash.gallery/Portal/
- https://dash.plotly.com/layout

Sidebar reference: https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Bootstrap/Side-Bar/side_bar.py 
Style references (via Dash Gallery):
- https://dash.gallery/dash-alignment-chart/
    - Layout Helper: https://github.com/plotly/dash-bio/blob/master/tests/dashbio_demos/dash-alignment-chart/layout_helper.py
    - CSS: https://github.com/plotly/dash-bio/blob/dc25279100c60de564945e351a316f2e91c63633/tests/dashbio_demos/dash-variant-map/assets/general-app-page.css

In [8]:
buttons_style = {    
    "text-decoration": "none",
    "font-size": "10pt",
    "font-family": "sans-serif",
    "color": "white",
    "border": "solid 1px white",
    "border-radius": "2px",
    "padding": "2px",
    "padding-top": "5px",
    "padding-left": "15px",
    "padding-right": "15px",
    "font-weight": "100",
    "position": "relative",
    "top": "15px",
    "float": "right",
    "margin-right": "20px",
    "margin-left": "0px",
    "transition-duration": "400ms"}

In [17]:
# HEADER

app_title = 'Al-Muhaddithat: The Women Narrators of Hadith'
bg_color = "#b593db"#"#a97ddb"
font_color = "#F3F6FA"


header = html.Div(
    id='app-page-header',
    children=[
        html.H2(
            app_title,
            style={"font-weight": "200 !important",
                    "letter-spacing": "1px",
                    "position": "absolute",
                    "top": "-5px",
                    "left":"5px",
                    "display": "inline-block"}
        ),
        
         html.Img(
            src='data:image/png;base64,{}'.format(
                base64.b64encode(
                    open(
                        './assets/GitHub-Mark-Light-64px.png',
                        'rb'
                    ).read()
                ).decode()
            ),
            style= {"height": "36px",
                    "float": "right",
                    "margin-top": "12px",
                    "margin-right": "15px",
                    "transition-duration": "500ms",
                    "margin-bottom":"12px"}
        ),
        
        html.A(
            id='gh-link',
            children=[
                'View on GitHub'
            ],
            href="https://github.com/muhaddithat/muhaddithat-networks", target="_blank", rel="noopener noreferrer",
             style=buttons_style
        ),

        html.A(
            id='user-manual-link',
            children=[
                'User Guide'
            ],
            href="https://github.com/muhaddithat/IN PROGRESS",
            style=buttons_style
        )

    ],
    style={
        'background': bg_color,
        'color': font_color,
         "width": "100%",
        "height": "60px !important",
        "position": "absolute",
        "top": "0px",
       
      
    }
)

In [10]:
# SIDEBAR with dropdown menu to select narrator + hadith/bios display

mainSidebar =   html.Div(style={  
    "position": "relative",
    "left": 0,
    "bottom": 0,
    "width": "18rem",
    "overflow-y":"auto",
    "height":'75vh', 
    "padding": "2vh 1vh",
    "background-color": "#f8f9fa",
},
             children=[
html.Div([
    # Dropdown to select a narrator
        dcc.Dropdown(
        id='dropdown1',
        options=[{'label': 'Aishah bint Abi Bakr', 'value': 53} ,
                 {'label': 'Hafsa bint Umar','value': 54},
                 {'label': 'Umm Salamah','value': 56},
                 {'label': 'Ramlah bint Abi Sufyan','value': 59},
                 {'label': 'Asmaa bint Umays','value': 69},
                 {'label': 'Asmaa bint Abi Bakr','value': 70},
                 {'label': 'Umm Atiyyah al-Ansariyyah','value': 71},
                 {'label': 'Asma bint Yazid ibn al-Sakan ','value': 84},
                 {'label': 'Zaynab bint Abi Salamah','value': 244},
                 {'label': 'Safiyyah bint Shayba','value': 2802},
                 {'label': 'Umm al-Darda as-Sughra','value': 11457},
                 {'label': 'Fatimah bint al-Munthir','value': 10526},
                 {'label': 'Khayra Umm al-Hasan al-Basri','value': 10737},             
                 {'label': 'Hafsah bint Sirin','value': 11039},],
        value=53,),], # set default to Aishah bint Abi Bakr, the woman who narrated the most hadiths
             ),
    
#     # Tabs to choose reading a bio or hadith associated with a narrator
#     html.Div(children=[
#         dcc.Tabs(id="tabs", value='tab-1', children=[
#             dcc.Tab(label='Bio', value='bio'),
#             dcc.Tab(label='Hadiths', value='hadiths'),
#         ]),
#         html.Div(id='tabs-content'),
#         ]), 
    
    html.Div(id='bio-display', className='?',
              children=[
         html.H4(children='Biography'),
         html.Div(id='bio-content')]),
                 
     html.Div(id='hadith-display', className='?',
              children=[
         html.H4(children='Hadith'),
         html.Div(id='hadith-content',children='')])
    ])

In [11]:
# right sidebar for info on clicked narrators
nodeSidebar =  html.Div(style={ 
                            "position": "absolute",
                            "top": 0,
                            "right": 0,
                            "width": "16rem",
                            "height":"16rem",
                            "padding": "2rem 1rem"
                            },
             children=[ html.H4("Narrator Info:"),
                        html.P(id='cytoscape-tapNodeData-output'),
                        html.P("Click on a node to learn about the narrator")
                    ])

# graph/cytoscape component for showing the network visualization    
graph = html.Div(children = [
    # cytoscape component:
    cyto.Cytoscape( 
        id='cytoscape',
        autoungrabify=True, # Prevent users from moving nodes around.
        elements= data['elements'], # edges + nodes
        layout={
            'name': 'breadthfirst',
            'directed': 'true',
            'maximal': 'true',
            'roots': '1', 
            'circle':'true'
        },
        style={'width':'77%', 'height':'90%', "top":"1vh", "left":"19rem",'position':'absolute', "justify-content":"center"}, 
        stylesheet=default_stylesheet
    ),
    
    # button to reset graph view:
    html.Button('reset', id='bt-reset', 
                style={
                    "background-color": "white",
                    "border": "solid 1px #d2b6f2",
                    "border-radius": "2px",
                    "padding": "2px",
                    "padding-top": "5px",
                    "padding-left": "15px",
                    "padding-right": "15px",
                    "font-weight": "100",
                    "padding-bottom":"5px",
                    "color": "#d2b6f2",
                    "text-align": "center",
                    "text-decoration": "none",
                    "display": "inline-block",
                    "font-size": "16px",
                    "position":"absolute",
                    "right":"19rem",
                    "top":"3rem"
    })])
    


In [18]:
app.layout = html.Div(style={"font-family":"Helvetica"}, 
                      
children=[
    html.Div(header), # app title + links to GitHub and user manual 
    
    html.Div(children=[
        mainSidebar, # sidebar for hadiths, bios, and dropdown for selecting a narrator

        nodeSidebar, # sidebar for displaying info about the clicked nodes

        graph # cytoscape graph

    ],
    style={"margin": "0px",
        "margin-top": "0px", # hide border between header and content
        "width": "100%",
        "height": "auto",
        "min-height": "calc(100vh - 60px)",
        "position": "absolute",
        "top": "60px",
    }
    ),
])

## Callbacks

The following functions update parts of the app (specified by the "output") based on different inputs (like the dropdown menu, clicking on a node, etc.).

References:
- https://dash.plotly.com/cytoscape/events
- https://dash.plotly.com/cytoscape/callbacks
- https://github.com/27shraddhaS/dropdown-graphs-dash/blob/main/dash-dropdown-graph.ipynb


In [13]:
# testing
# value = 1
# "testing{}".format(str(value))
# "[source={}{}{}]".format("'", value, "''")

In [14]:
# For nodeSidebar - display brief information on a clicked node 
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    if data:
        gender = "female" if data["gender"] == 'f' else "male"
        newline = '\n'
#         return "Name: " + data['name'] + "\nGender: " + gender + "\nGeneration: " + data["generation"] 
#         return f"Name: {data['name']}{newline}\nGender: {gender}{newline}\nGeneration: {data['generation']}"
        return '''
        Name: {}
        
        Gender: {}
        
        Generation: {}
        '''.format(data['name'], gender, data['generation'])

          
    
# Update the cytoscape element so it displays the graph associated with the narrator selected in the dropdown menu:

@app.callback(Output('cytoscape', 'elements'),
              Input('dropdown1', 'value'))

def update_elements(value): 
    # open the file corresponding to the selected narrator's graph
    with open('data/subgraphs/SUBGRAPH_TO-ROOT_MAX-DEPTH-1_SEEDS-{}.cyjs'.format(str(value)), 'r') as f:
        data = json.load(f)
    return data['elements']
    
# Update the stylesheet so that the edges of the selected narrator are colored 

@app.callback(Output('cytoscape', 'stylesheet'),
              Input('dropdown1', 'value'))

def update_stylesheet(value):  
    
    selector = "[source={}{}{}]".format("'", value, "'") # specify which ID we want to color differently
    
    new_styles = [{
        'selector': 'edge',
        'style': {
            'line-color': '#7b7b7b',
            'width':'0.1px',
            'opacity':'0.2',
            'curve-style': 'bezier',
            'target-arrow-shape': 'vee',
            'target-arrow-fill': 'hollow',
            'arrow-scale':'0.1'
        }
    },  
            {
          'selector': selector, #"[source='53']",
          'style':{
          'line-color':'#ff0000',
          'width':'0.1px',
          'opacity':'1',
          'target-arrow-color':'#ff0000'} 
        }]
         

    return default_stylesheet + new_styles

# Display the hadiths associated with the selected narrator from the dropdown menu:

@app.callback(Output('hadith-content', 'children'),
            Input('dropdown1', 'value'))
def update_hadith_display(value):
    #return hadiths[0]['EnglishText']#"hello " + str(value)
    for h in hadiths:
        if value in h['NarrationChain']:
            return(h['EnglishText'])

# idea:
# maybe make a list or dictionary that maps each narrator ID with their hadiths so you can access them easily
# instead of using the following for loop that doesn't display right.

# Display the bio associated with the selected narrator from the dropdown menu:

@app.callback(Output('bio-content', 'children'),
            Input('dropdown1', 'value'))
def update_bio_display(value):
    #return bios[0]['Biography']
    for b in bios:
        if b['NarratorID'] == value:
            return b['Biography']

# Reset view/zoom of the graph when the reset button is clicked
# Reference: https://github.com/plotly/dash-cytoscape/blob/master/demos/usage-reset-button.py

@app.callback(
    [Output('cytoscape', 'zoom'),
     Output('cytoscape', 'elements')],
    [Input('bt-reset', 'n_clicks')]
)
def reset_layout(n_clicks):
#    print(n_clicks)
    return [1, data['elements']] #[1, elements]

# # TABS
# @app.callback(Output('tabs-content', 'children'),
#               Input('tabs', 'value')) # issue: the tabs do not update whenever a new narrator is selected from the dropdown. perhaps another callback is needed.
# def render_content(tab):
#     if tab == 'bio':
#         for b in bios:
#             if b['NarratorID'] == currentNarrator:
#                 return html.Div([
#                     b['Biography'] # issue: it doesn't read the \n characters
#                 ])
#     elif tab == 'hadiths':
#         for h in hadiths:
#             if currentNarrator in h['NarrationChain']:
#                 return html.Div([
#                     h['EnglishText'] # issue: it currently just returns after seeing the first hadith, fix so that it shows ALL of them - it must run the whole for loop before returning
#                 ])

# @app.callback(Output('tabs-content', 'children'),
 #             Input('dropdown1', 'value'))

    


In [15]:
# testing
# for h in hadiths:
#     print(h['NarrationChain'])
#     if currentNarrator in h['NarrationChain']:
#         print(h['EnglishText'])
# this works so idk why it doesn't return on the app


## Run App

In [16]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
